In [1]:
!pip install langchain chromadb sentence-transformers huggingface_hub transformers torch==2.6.0 torchvision==0.21.0 zipfile36 langchain-community langchain-huggingface gradio -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.8 MB/s 

In [2]:
import os
import zipfile
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_huggingface  import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from getpass import getpass

In [3]:
# Set Hugging Face API key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter your Hugging Face API key: ")

Enter your Hugging Face API key: ··········


In [4]:
# Download the ZIP file
url = "https://www.dropbox.com/scl/fi/p7d0zjnrefjne94j3ec2b/new_articles.zip?rlkey=izy5z44eofgx5fmjn8tif4y99&dl=1"
response = requests.get(url)
with open("/content/new_articles.zip", "wb") as f:
    f.write(response.content)

# Extract ZIP file
with zipfile.ZipFile("/content/new_articles.zip", "r") as zip_ref:
    zip_ref.extractall("/content/articles")

# Read and preprocess text files
articles = []
for file_name in os.listdir("/content/articles"):
    if file_name.endswith(".txt"):
        with open(f"/content/articles/{file_name}", "r", encoding="utf-8") as f:
            content = f.read().strip()
            articles.append({"content": content, "filename": file_name})

In [5]:
articles[1]

{'content': 'Slack has evolved from a pure communications platform to one that enables companies to link directly to enterprise applications without having to resort to dreaded task switching. Today, at the Salesforce World Tour event in NYC, the company announced the next step in its platform’s evolution where it will be putting AI at the forefront of the user experience, making it easier to get information and build workflows.\n\nIt’s important to note that these are announcements, and many of these features are not available yet.\n\nRob Seaman says that rather than slapping on an AI cover, they are working to incorporate it in a variety of ways across the platform. That started last month with a small step, a partnership with OpenAI to bring a ChatGPT app into Slack, the first piece of a much broader vision for AI on the platform. That part is in beta at the moment.\n\nToday’s announcement involves several new integrations, including SlackGPT, the company’s own flavor of generative 

In [6]:
# Chunk the articles
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = []
for article in articles:
    split_texts = splitter.split_text(article["content"])
    for i, text in enumerate(split_texts):
        chunks.append({
            "text": text,
            "metadata": {"filename": article["filename"], "chunk_id": i}
        })

In [7]:
chunks[1]

{'text': 'I won’t encumber the reader with a lengthy summary of this perfectly readable and interesting piece, but the gist is that while GPT-4 and other proprietary models have obtained the lion’s share of attention and indeed income, the head start they’ve gained with funding and infrastructure is looking slimmer by the day.\n\nWhile the pace of OpenAI’s releases may seem blistering by the standards of ordinary major software releases, GPT-3, ChatGPT and GPT-4 were certainly hot on each other’s heels if you compare them to versions of iOS or Photoshop. But they are still occurring on the scale of months and years.',
 'metadata': {'filename': '05-05-google-and-openai-are-walmarts-besieged-by-fruit-stands.txt',
  'chunk_id': 1}}

In [8]:
# Set up embeddings and ChromaDB
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_texts(
    texts=[chunk["text"] for chunk in chunks],
    embedding=embeddings,
    metadatas=[chunk["metadata"] for chunk in chunks],
    persist_directory="/content/chroma_db"
)

<ipython-input-8-3800308b58a8>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Set up FLAN-T5 model
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("Model loaded")
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    device= 0  # Use GPU
)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


Model loaded


In [10]:
# Create RetrievalQA chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [11]:
# Test the chatbot
query1 = "What is the main topic of the articles?"
print("Query 1:", query1)
result1 = qa_chain.invoke({"query": query1})
print("Answer 1:", result1["result"])

query2 = "What did Pando company do?"
print("Query 2:", query2)
result2 = qa_chain.invoke({"query": query2})
print("Answer 2:", result2["result"])

query3 = "Who did Databricks acquire?"
print("Query 3:", query3)
result3= qa_chain.invoke({"query": query3})
print("Answer 3:", result3["result"])

# print("\nSources:")
# for doc in result["source_documents"]:
#     print(f"- {doc.metadata['filename']} (Chunk {doc.metadata['chunk_id']})")

Token indices sequence length is longer than the specified maximum sequence length for this model (1037 > 512). Running this sequence through the model will result in indexing errors


Query 1: What is the main topic of the articles?
Answer 1: AI
Query 2: What did Pando company do?
Answer 2: solve for global logistics through a software-as-a-service platform offering
Query 3: Who did Databricks acquire?
Answer 3: Okera


In [15]:
import gradio as gr

# Define the QA function using your qa_chain
def answer_question(question):
    if not question.strip():
        return "Please enter a question."
    try:
        response = qa_chain.invoke({"query": question})
        return response["result"]
    except Exception as e:
        return f"Error: {str(e)}"

# Launch the Gradio interface
gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(label="Question", placeholder="Ask your question here...", lines=1),
    outputs=gr.Textbox(label="Answer"),
    title="News Article QA Chatbot",
    description="Ask questions based on the article collection.",
    theme="default",
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e0b0eafef51f44ab2c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
